# <span style="color:gray">Pangolin SARS-CoV-2 Pipeline Notebook</span>

We are going to run a standard covid bioinformatics pipeline using the Pangolin workflow. https://cov-lineages.org/resources/pangolin/usage.html

### Required software

In [1]:
#change this depending on how many threads are available in your notebook
CPU=4

In [2]:
%%bash

#install Mamba
curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge
rm Mamba*

PREFIX=/home/ec2-user/mambaforge
Unpacking payload ...
Extracting "libmambapy-0.19.0-py39h8bfa403_0.tar.bz2"
Extracting "zstd-1.5.0-ha95c52a_0.tar.bz2"
Extracting "readline-8.1-h46c0cb4_0.tar.bz2"
Extracting "yaml-cpp-0.6.3-he1b5a44_4.tar.bz2"
Extracting "libgcc-ng-11.2.0-h1d223b6_11.tar.bz2"
Extracting "cffi-1.15.0-py39h4bc2ebd_0.tar.bz2"
Extracting "wheel-0.37.0-pyhd8ed1ab_1.tar.bz2"
Extracting "colorama-0.4.4-pyh9f0ad1d_0.tar.bz2"
Extracting "_openmp_mutex-4.5-1_gnu.tar.bz2"
Extracting "tzdata-2021e-he74cb21_0.tar.bz2"
Extracting "reproc-cpp-14.2.3-h9c3ff4c_0.tar.bz2"
Extracting "lz4-c-1.9.3-h9c3ff4c_1.tar.bz2"
Extracting "libarchive-3.5.2-hccf745f_1.tar.bz2"
Extracting "libedit-3.1.20191231-he28a2e2_2.tar.bz2"
Extracting "icu-69.1-h9c3ff4c_0.tar.bz2"
Extracting "ld_impl_linux-64-2.36.1-hea4e1c9_2.tar.bz2"
Extracting "libzlib-1.2.11-h36c2ea0_1013.tar.bz2"
Extracting "tqdm-4.62.3-pyhd8ed1ab_0.tar.bz2"
Extracting "ncurses-6.2-h58526e2_4.tar.bz2"
Extracting "openssl-1.1.1l-h7f98852_0.t

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   160  100   160    0     0    969      0 --:--:-- --:--:-- --:--:--   969
100   665  100   665    0     0   2224      0 --:--:-- --:--:-- --:--:--  2224
100  102M  100  102M    0     0  14.3M      0  0:00:07  0:00:07 --:--:-- 14.4M
bash: line 7: !cp: command not found


In [20]:
#move mamba executable to your path
!cp ~/mambaforge/bin/mamba /home/ec2-user/anaconda3/condabin

In [21]:
#install biopython to import packages below
!pip install biopython

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.


In [41]:
!conda install ipyrad -y -c conda-forge -c bioconda

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::seaborn-base==0.11.1=pyhd8ed1ab_1
  - conda-forge/noarch::nbclassic==0.2.6=pyhd8ed1ab_0
  - conda-forge/noarch::typing-extensions==3.7.4.3=0
  - conda-forge/linux-64::pluggy==0.13.1=py36h5fab9bb_4
  - conda-forge/linux-64::blaze==0.11.3=py36_0
  - conda-forge/linux-64::matplotlib==3.3.4=py36h5fab9bb_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - conda-forge/linux-64::scikit-image==0.16.2=py36hb3f55d8_0
  - conda-forge/noarch::path.py==12.5.0=0
  - conda-forge/noarch::qdarkstyle==2.8.1=pyhd8ed1ab_2
  - conda-forge/noarch::ipywidgets==7.6.3=pyhd3deb0d_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/linux-64:

Now we want to create a conda/mamba env that has all of our necessary dependencies

In [23]:
#you can look at the yaml file that specifies which programs we want to install
#you can also specify specific versions, here we just use the latest conda versionå
#for example, - sra-tools=2.11.0
!cat pangolin.yaml

name: pangolin
channels:
  - bioconda
  - conda-forge
  - defaults
  - eaton-lab
  
dependencies:
  - sra-tools
  - ipyrad
  - toytree
  - pangolin
  - iqtree


In [25]:
#create the environment. Here we use mamba because it is faster than conda
!mamba env create -f pangolin.yaml -y

usage: mamba [-h] {create,export,list,remove,update,config} ...
mamba: error: unrecognized arguments: -y


In [33]:
#give it the whole path to the env because otherwise it can't find the env
#if you want to play with it add a cell and type 'conda activate pangolin' 
#or 'source activate pangolin'
!source activate /home/ec2-user/mambaforge/envs/pangolin
#!mamba info --envs

In [35]:
!mamba install -c bioconda iqtree -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.19.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['iqtree']

bioconda/linux-64        Using cache
bioconda/noarch          Using cache
conda-forge/l

In [40]:
#import libraries
import os
from Bio import SeqIO
from Bio import Entrez
import ipyrad.analysis as ipa
import toytree

ModuleNotFoundError: No module named 'ipyrad'

### Set up your directory structure and remove files from previous runs if they exist

In [28]:
cd /home/jupyter/cloud-lab-training/GCP/notebooks/pangolin/

/home/jupyter/cloud-lab-training/GCP/notebooks/pangolin


In [29]:
if not os.path.exists('pangolin_analysis'):
    os.mkdir('pangolin_analysis')
os.chdir('pangolin_analysis')

In [30]:
if os.path.exists('sarscov2_sequences.fasta'):
    os.remove('sarscov2_sequences.fasta')
!rm sarscov2_*
!rm lineage_report.csv

rm: cannot remove 'sarscov2_*': No such file or directory
rm: cannot remove 'lineage_report.csv': No such file or directory


### Fetch viral sequences using a list of accession IDs

In [31]:
#give a list of accession number for covid sequences
acc_nums=['NC_045512','LR757995','LR757996','OL698718','OL677199','OL672836','MZ914912','MZ916499','MZ908464','MW580573','MW580574','MW580576','MW991906','MW931310','MW932027','MW424864','MW453109','MW453110']
print('the number of sequences we will analyze = ',len(acc_nums))

the number of sequences we will analyze =  18


Let this block run without going to the next until it finishes, otherwise you may get an error about too many requests. If that happens, reset your kernel and just rerun everything (except installing software).

In [32]:
#use the bio.entrez toolkit within biopython to download the accession numbers
#save those sequences to a single fasta file
Entrez.email = "email@example.com"  # Always tell NCBI who you are
filename = "sarscov2_seqs.fasta"
if not os.path.isfile(filename):
    # Downloading...
    for acc in acc_nums:
        net_handle = Entrez.efetch(
            db="nucleotide", id=acc, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "a")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        print("Saved",acc)

Saved NC_045512
Saved LR757995
Saved LR757996
Saved OL698718
Saved OL677199
Saved OL672836
Saved MZ914912
Saved MZ916499
Saved MZ908464
Saved MW580573
Saved MW580574
Saved MW580576
Saved MW991906
Saved MW931310
Saved MW932027
Saved MW424864
Saved MW453109
Saved MW453110


In [15]:
#make sure our fasta file has the same number of seqs as the acc_nums list
print('the number of seqs in our fasta file: ')
!grep '>' sarscov2_seqs.fasta | wc -l

the number of seqs in our fasta file: 
18


In [16]:
#let's peek at our new fasta file
!head sarscov2_seqs.fasta

>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT
GCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTC
GTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCT


### Run pangolin to identify lineages and output alignment
Here we call pangolin, give it our input sequences and the number of threads. We also tell it to output the alignment. The full list of pangolin parameters can be found in the [docs](https://cov-lineages.org/resources/pangolin/usage.html).

In [33]:
!pangolin sarscov2_seqs.fasta --alignment --threads $CPU

All dependencies satisfied.
The query file is:/home/jupyter/cloud-lab-training/GCP/notebooks/pangolin/pangolin_analysis/sarscov2_seqs.fasta
** Running sequence QC **
Number of sequences detected: 18
Total passing QC: 18

Data files found:
Trained model:	/opt/conda/lib/python3.7/site-packages/pangoLEARN/data/decisionTree_v1.joblib
Header file:	/opt/conda/lib/python3.7/site-packages/pangoLEARN/data/decisionTreeHeaders_v1.joblib
Designated hash:	/opt/conda/lib/python3.7/site-packages/pangoLEARN/data/lineages.hash.csv
Job stats:
job                     count    min threads    max threads
--------------------  -------  -------------  -------------
add_failed_seqs             1              1              1
align_to_reference          1              1              1
all                         1              1              1
generate_report             1              1              1
get_constellations          1              1              1
hash_sequence_assign        1              1     

You can view the output file from pangolin called lineage_report.csv (within pangolin_analysis folder) by double clicking on the file, or by right clicking and downloading. What lineages are present in the dataset? Is Omicron in there?

### Run iqtree to estimate maximum likelihood tree for our sequences
iqtree can find the best nucleotide model for the data, but here we are going to assign a model to save time (HKY) and just estimate the phylogeny without any bootstrap support values. 

In [18]:
#run iqtree with threads = $CPU variable, if you exclude the -m it will do a phylogenetic model search before tree search
!iqtree -s sequences.aln.fasta -nt $CPU -m HKY --prefix sarscov2_tree --redo-tree

IQ-TREE multicore version 2.1.4-beta COVID-edition for Linux 64-bit built Jun 24 2021
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung,
Olga Chernomor, Heiko Schmidt, Dominik Schrempf, Michael Woodhams.

Host:    cloud-lab-notebook (AVX2, FMA3, 14 GB RAM)
Command: iqtree -s sequences.aln.fasta -nt 4 -m HKY --prefix sarscov2_tree --redo-tree
Seed:    719057 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Fri Dec  3 23:53:05 2021
Kernel:  AVX+FMA - 4 threads (4 CPU cores detected)

Reading alignment file sequences.aln.fasta ... Fasta format detected
Alignment most likely contains DNA/RNA sequences
Alignment has 18 sequences with 29903 columns, 193 distinct patterns
109 parsimony-informative, 33 singleton sites, 29761 constant sites
LR757995.1_Severe_acute_respiratory_syndrome_coronavirus_2_genome_assembly__chromosome:_whole_genome -> LR757995.1_Severe_acute_respiratory_syndrome_coronavirus_2_genome_assembly__chromosome__whole_genome
LR757996.1_Severe_acute_

### Visualize the tree with toytree

In [19]:
#Define the tree file
tre = toytree.tree('sarscov2_tree.treefile')

In [23]:
#draw the tree
rtre = tre.root(wildcard="OL")
rtre.draw(tip_labels_align=True);

<svg class="toyplot-canvas-Canvas" height="324.0px" id="t854aace90f344035b0524662eb5e8bd5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 324.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> MW453110.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/CA-LACPHL-AF00093/2020__complete_genome MW453109.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/CA-LACPHL-AF00094/2020__complete_genome MW424864.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/CA-LACPHL-AF00051/2020__complete_genome MW580576.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/MD-MDH-0833/2021__complete_genome MW580574.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/MD-MDH-0831/2021__complete_genome MW580573.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/MD-MDH-0830/2021_ORF1ab_polyprotein__ORF1ab___ORF1a_polyprotein__ORF1ab___surface_glycoprotein__S___ORF3a_protein__ORF3a___envelope_protein__E___membrane_glycoprotein__M___ORF6_protein__ORF6___ORF7a_protein__ORF7a___ORF7b__ORF7b___ORF8_protein__ORF8___nucleocapsid_phosphoprotein__N___and_ORF10_protein__ORF10__genes__complete_cds MZ908464.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/TG769681/2020_ORF1ab_polyprotein__ORF1ab___ORF1a_polyprotein__ORF1ab___surface_glycoprotein__S___ORF3a_protein__ORF3a___envelope_protein__E___membrane_glycoprotein__M___and_ORF6_protein__ORF6__genes__complete_cds__ORF7a_protein__ORF7a__and_ORF7b__ORF7b__genes__partial_cds__and_ORF8_protein__ORF8___nucleocapsid_phosphoprotein__N___and_ORF10_protein__ORF10__genes__complete_cds MZ914912.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/TG796484/2020_ORF1ab_polyprotein__ORF1ab___ORF1a_polyprotein__ORF1ab___surface_glycoprotein__S___ORF3a_protein__ORF3a___envelope_protein__E___membrane_glycoprotein__M___ORF6_protein__ORF6___ORF7a_protein__ORF7a___ORF7b__ORF7b___ORF8_protein__ORF8___nucleocapsid_phosphoprotein__N___and_ORF10_protein__ORF10__genes__complete_cds MZ916499.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/TG841289/2020_ORF1ab_polyprotein__ORF1ab__and_ORF1a_polyprotein__ORF1ab__genes__partial_cds__and_surface_glycoprotein__S___ORF3a_protein__ORF3a___envelope_protein__E___membrane_glycoprotein__M___ORF6_protein__ORF6___ORF7a_protein__ORF7a___ORF7b__ORF7b___ORF8_protein__ORF8___nucleocapsid_phosphoprotein__N___and_ORF10_protein__ORF10__genes__complete_cds LR757996.1_Severe_acute_respiratory_syndrome_coronavirus_2_genome_assembly__chromosome__whole_genome NC_045512.2_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_Wuhan-Hu-1__complete_genome LR757995.1_Severe_acute_respiratory_syndrome_coronavirus_2_genome_assembly__chromosome__whole_genome MW932027.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/MA-CDC-STM-000044850/2021_ORF1ab_polyprotein__ORF1ab___ORF1a_polyprotein__ORF1ab___surface_glycoprotein__S___ORF3a_protein__ORF3a___envelope_protein__E___membrane_glycoprotein__M___ORF6_protein__ORF6___ORF7a_protein__ORF7a___and_ORF7b__ORF7b__genes__complete_cds__ORF8_gene__complete_sequence__and_nucleocapsid_phosphoprotein__N__and_ORF10_protein__ORF10__genes__complete_cds MW991906.1_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_SARS-CoV-2/human/USA/CA-CDC-FG-021330/2021_ORF1ab_polyprotein__ORF1ab__and_ORF1a_polyprotein__ORF1ab__genes__partial_cds__surface_glycoprotein__S___ORF3a_protein__ORF3a___envelope_protein__E___membrane_glycoprotein__M___ORF6_protein__ORF6___ORF7a_protei

You can also visualize the tree by downloading it and opening in figtree.

And that is all! You now know how to run workflows in notebooks in Cloud Lab